<a href="https://colab.research.google.com/github/crystalloide/Spark/blob/master/Big_Data_and_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Big Data

Le Big Data n'est que de la donnée, mais en quantité ?

Le Big Data peut être décrit à l'aide des «quatre V» (on évoque même 5 ou 7 V). 

Les quatre premiers V sont les suivants :
1. Volume
2. Vitesse
3. Variété
4. Véracité

Les mégadonnées sont décrites comme étant volumineuses, en volume donc (quantité de données : de l'ordre de zétaoctets par exemple), comme arrivant à grande vitesse (données en streaming, par exemple, les données des capteurs ou des téraoctets d'informations commerciales), comme étant de différentes variétés (différentes formes de données, par exemple les vidéos et les tweets) et comme étant d'une véracité plus ou moins fiable, on parle alors d'incertitude de la valeur des données (par exemple, mauvaise qualité des données).

En raison de la charge de traitement des mégadonnées, il est nécessaire d'utiliser des outils spéciaux optimisés pour des calculs de cette taille.



# Clusters de Stockage et de Calcul

Un seul ordinateur ne sera pas en mesure de stocker seul l'ensemble des données à traiter. C'est ce qui est appelé un cluster de stockage : un groupe d'ordinateurs qui se répartissent les fragments de fichiers pour faire stocker la totalité des informations à traiter.

L'overhead du traitement des données Big Data  provient du fait qu'un seul ordinateur ne fera pas le travail de traitement des données «big data», mais il faudra faire fonctionner plusieurs ordinateurs ensemble. 
C'est ce qui est appelé un cluster de calcul : un groupe d'ordinateurs qui travaillent ensemble pour faire le travail attendu.

Comment gérer un ensemble d'ordinateurs pour travailler ensemble afin d' accomplir une tâche? 

Cette gestion du travail en cluster est en réalité difficile à gérer, du fait de la concurrence, de la communication interprocessus, de la planification des tâches, etc. A cela s'ajoute les problèmes "normaux" rencontrés sur des systèmes distribués comme les pannes d'ordinateur ou la latence du réseau, etc.

# Hadoop

Pour permettre ces usages liés au Big Data, ** Apache Hadoop ** a été mis au point et est constitué d'un framework, une collection d'outils, destinés à gérer les clusters et les utiliser.

- Yarn : gestion des tâches de calcul dans le cluster
- HDFS (Hadoop Distributed File System) : stockage des données réparties sur les nœuds du cluster (ordinateurs ou nodes)
- Spark : framework permettant de réaliser des calculs sur les données

# Débutons avec Spark
1. Télécharger [Spark (2.4.3)](https://spark.apache.org/) (ou la version pre-built la plus récente)
2. Positionner une variable d'environnement (avec un terminal). Si on utilise  Python 3, cela donne :
> export PYSPARK_PYTHON=python3
3. Positionner également le path : (chemin d'accès vers les binaires)
> export PATH=${PATH}:/home/you/spark-2.4.4-bin-hadoop2.7/bin
4. Si vous obtenez une  erreur 'Py4JJavaError', vous devrez installer Java ou OpenJDK version 8

Voilà comment configurer Spark.

De plus, Pour exécuter un programme spark, il suffit ensuite d'entrer la commande suivante en ligne de commande dans une terminal :

> spark-submit spark-program.py

# Notre 1er programme Spark

Il faut ensuite cliquer sur "Ajouter des données" en haut à droite et de choisir "Los Angeles Traffic Collision Dataset".

A noter : 
N'hésitez pas à utiliser un autre dataset lors de futures expériences. 
Il faut alors vérifiez le type de fichier et changer la méthode de lecture spark en conséquence.

In [0]:
import os
input_dir = '../input'
os.listdir(input_dir)
file = 'traffic-collision-data-from-2010-to-present.csv'
path = os.path.join(input_dir,file)
print(path)

In [0]:
!pip install pyspark

In [0]:
import sys
from pyspark.sql import SparkSession, functions, types
 
spark = SparkSession.builder.appName('example 1').getOrCreate()
spark.sparkContext.setLogLevel('WARN')

assert sys.version_info >= (3, 5) # make sure we have Python 3.5+
assert spark.version >= '2.3' # make sure we have Spark 2.3+

data = spark.read.csv(path, header=True,
                      inferSchema=True)
data.show()

Yeah.. it doesn't look pretty. Let's see what we can do. First, we explore some methods with a Spark dataframe.

In [0]:
# let's see the schema
data.printSchema()

In [0]:
# select some columns
data.select(data['Crime Code'], data['Victim Age']).show()

In [0]:
# filter the data
data.filter(data['Victim Age'] < 40).select('Victim Age', 'Victim Sex').show()

In [0]:
# write to a json file
json_file = data.filter(data['Victim Age'] < 40).select('Victim Age', 'Victim Sex')
json_file.write.json('json_output', mode='overwrite')

If you were expecting one json file well no, instead you get a **directory** of multiple json files. The concatenation of those files is the actual output. This is because of the way Spark computes. More on it later.

In [0]:
!ls json_output

In [0]:
# a few more things

# perform a calculation on a column and rename it
data.select((data['Council Districts']/2).alias('CD_dividedBy2')).show()

# rename columns 
data.withColumnRenamed('Victim Sex', 'Gender').select('Gender').show()

# drop columns and a cleaner vertical format for the top 10 
d = data.drop('Neighborhood Councils')
d.show(n=10, truncate=False, vertical=True)

# Partitioning
We previously saw the output of json file is resulted with a directory of multiple json files. This is because we said that big data is too big to be processed on one single computer which is why the Apache Hadoop toolset is there to be able to work with data and compute on multiple computers but can come together as one result as if the data was processed on one machine. This is why all Spark dataframes are partitioned this way no matter how small the data is. 

Ususally you would give an input directory of files as our "data" where each thread/process/core/executor reads an indvidual input file. When creating the output, each write is done in parallel and when each of the output files are combined they form the single output result. That is where HDFS plays a part as the shared filesystem for all of this parallelism to work. 

YARN is responsbile for managing the computation on each individual computer when actally working with a cluster of nodes. YARN manages the CPU and memory resources. Rather than moving the data to different nodes, YARN can move the compute work to where the data is.

On the local machine, we just use the local filesytem

# Conclusion
This was a very breif overview of Big Data and Spark. I am just studying for my final so I thought might as well write about it and share the knowledge with others as a way of studying. If this was useful let me know and I will continue with more details on more PySpark stuff like how it calculates, grouping data and joining data. Also I am not an expert on this stuff so if I am giving some wrong information let me know.  